In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
class Dynamics:
    pass

In [3]:
hw2dynamics=Dynamics()
hw2dynamics.S0 = 100
hw2dynamics.r = 0.06
hw2dynamics.q = 0.01
hw2dynamics.maxvol = 0.6
hw2dynamics.localvol = lambda S,t: np.minimum(0.2+5*np.log(S/100)**2+0.1*np.exp(-t), 0.6)

# Note that hw2dynamics.localvol is a function 
# that may be invoked in the usual way, for example:
# hw2dynamics.localvol( exchangerate , time )

In [4]:
class Contract:
    pass

In [5]:
hw2contract=Contract()
hw2contract.putexpiry = 0.75;
hw2contract.putstrike = 95;
hw2contract.callexpiry = 0.25;
hw2contract.callstrike = 10;

In [6]:
class Tree:
    pass

In [7]:
hw2tree=Tree()
hw2tree.N=60000  #change if necessary to get $0.01 accuracy, in your judgment

In [8]:
# You complete the coding of this function

def pricer_compound_localvol_trinom(contract,dynamics,tree):
    def calc_dx(avg_sig, max_sig, deltat):
        return max(avg_sig * (3*deltat)**0.5, max_sig * (deltat)**0.5)
    
    def calc_nu(r, sigma):
        return r-sigma**2/2

    def calc_Ps(sigma, nu, deltat, deltax):
        inside = deltat * (sigma**2 + nu**2 * deltat) / deltax**2
        back = nu * deltat / deltax
        Pu = 1/2 * (inside + back)
        Pd = 1/2 * (inside - back)
        Pm = 1 - inside
        return Pu, Pd, Pm
    
    def back_induct(r, deltat, Pu, Pd, Pm, Cu, Cd, Cm):
        return np.exp(-r*deltat) * (Pu*Cu + Pd*Cd + Pm*Cm)
    
    S0, r, q, max_sig, localvol = dynamics.S0, dynamics.r, dynamics.q, dynamics.maxvol, dynamics.localvol
    putT, putK, callT, callK = contract.putexpiry, contract.putstrike, contract.callexpiry, contract.callstrike
    N = tree.N
    
    deltat = putT/N # Fill this in with a scalar.
    avg_sig = localvol(S0, 0)
    deltax = calc_dx(avg_sig, max_sig, deltat)  #Fill this in with a scalar.
    
    S=S0*np.exp(np.linspace(N, -N, num=2*N+1, endpoint=True)*deltax)
        
    putV = np.maximum(putK-S,0)   #an array of time-T option prices.    
        
    #Next, induct backwards to time 0, updating the optionprice array 
    #Hint: if x is an array, then what are x[2:] and x[1:-1] and x[:-2]
    timelineP = np.linspace(N-1, 0, num=N, endpoint=True)*deltat
    callT_less1 = -1
    for t in timelineP:
        S = S[1:-1]
        sigma = localvol(S, t)
        nu = calc_nu(r-q, sigma) # complete this 
        Pu, Pd, Pm = calc_Ps(sigma, nu, deltat, deltax)
        
        # insert lines of code here if needed
        putV = back_induct(r, deltat, Pu, Pd, Pm, putV[:-2], putV[2:], putV[1:-1])   #complete this
        putV = np.maximum(putV, putK-S)
        
        # Steal put prices @ callT to price call
        if abs(t-callT) < deltat/2:
            callV = np.maximum(putV-callK, 0)#putV
            callS = S
            callT_less1 = t
            
        if t < callT_less1:
            callV = back_induct(r, deltat, Pu, Pd, Pm, callV[:-2], callV[2:], callV[1:-1])
    
    price_of_put = putV[0] #write code to compute this
    price_of_call_on_put = callV[0] #write code to compute this
    
    return (price_of_put, price_of_call_on_put)

# Problem 1

The Gold Dragon Coin (GDC) is a unit of currency in Westeros. Let S denote the GDC/USD exchange rate (the USD value of 1 GDC). Assume S has dynamics...

In [9]:
(answer_part_a, answer_part_b) = pricer_compound_localvol_trinom(hw2contract,hw2dynamics,hw2tree)
print(f"1(a) American Put on GDC: {answer_part_a:.4f}")
print(f"1(a) European Call on GDC Put: {answer_part_b:.4f}")

1(a) American Put on GDC: 7.0073
1(a) European Call on GDC Put: 1.5924


In [10]:
def BScallPrice(sigma,S,rGrow,r,K,T):
    F=S*np.exp(rGrow*T)
    sd = sigma*np.sqrt(T)
    d1 = np.log(F/K)/sd+sd/2
    d2 = d1-sd
    return np.exp(-r*T)*(F*norm.cdf(d1)-K*norm.cdf(d2))

def BSputPrice(sigma,S,rGrow,r,K,T):
    F=S*np.exp(rGrow*T)
    sd = sigma*np.sqrt(T)
    d1 = np.log(F/K)/sd+sd/2
    d2 = d1-sd
    return np.exp(-r*T)*K*norm.cdf(-d2)-F*norm.cdf(-d1)

sigma = 0.3
S = hw2dynamics.S0
rGrow = 0
r = hw2dynamics.r
K = hw2contract.putstrike
T = hw2contract.putexpiry

lower = BSputPrice(sigma,S,rGrow,r,K,T)
sigma = 0.6
upper = BSputPrice(sigma,S,rGrow,r,K,T)

print(f"Put lower Bound: {lower:.4f}")
print(f"Put upper Bound: {upper:.4f} (neglecting early exercise)")

Put lower Bound: 5.7944
Put upper Bound: 15.2238 (neglecting early exercise)


# Problem 2

(a) 

$$\begin{aligned}
    \Delta &= \frac{dV}{dS} = \Phi(d_1) \\
    d_1 &= \frac{1}{\sigma \sqrt{T}} \bigg[ log(\frac{S}{K})+(r+\frac{\sigma^2}{2})(T) \bigg] \\
        &= \frac{1}{\sigma \sqrt{T}} \bigg[ (r+\frac{\sigma^2}{2})(T) \bigg] \\
    \\
    d_1 &\to 0: \\
    \Phi(d_1) &= \Phi(0)+\phi(0)d_1+\frac{1}{2}\phi'(0)d_1^2+O(d_1^3) \\
        &\approx \frac{1}{2}+\frac{1}{\sqrt{2\pi}}d_1+0+0 \\
        &= \frac{1}{2}+\frac{1}{\sqrt{2\pi}}\bigg( \frac{1}{\sigma \sqrt{T}} \bigg[ (r+\frac{\sigma^2}{2})(T) \bigg] \bigg) \\
        &= \frac{1}{2}+\frac{1}{\sqrt{2\pi}}\bigg( \frac{1}{0.2 \sqrt{0.25}} \bigg[ (0.01+\frac{0.2^2}{2})(0.25) \bigg] \bigg) \\
        &= \frac{1}{2}+\frac{1}{\sqrt{2\pi}}\bigg( 10 \bigg[ (0.01+0.03)(0.25) \bigg] \bigg) \\
        &= \frac{1}{2}+\frac{2}{5}(0.075) \\
        &= 0.53
\end{aligned}$$

(b)

$$\begin{aligned}
    C(S+\Delta S) &= C(S) + \frac{dC}{dS}(S)(\Delta S) + \frac{1}{2}\frac{d^2C}{dS^2}(S)(\Delta S)^2 + O(\Delta S^3)... \\
    \\
    S_0 &= 4, C(S_0)=5, S_0\frac{dC}{dS}(S_0) = 3, \frac{S_0^2}{100}\frac{d^2C}{dS^2}(S_0) = 0.02 \\
    \frac{dC}{dS}(S_0) &= 0.75, \frac{d^2C}{dS^2}(S_0) = 0.125 \\
    \\
    \Delta S &= -0.4 \\
    C(S_0+\Delta S) &\approx C(S_0) + \frac{dC}{dS}(S_0)(\Delta S) + \frac{1}{2}\frac{d^2C}{dS^2}(S_0)(\Delta S)^2 \\
    C(4-0.4) &\approx C(4) + \frac{dC}{dS}(4)(-0.4) + \frac{1}{2}\frac{d^2C}{dS^2}(4)(-0.4)^2 \\
        &= 5 + 0.75 (-0.4) + \frac{1}{2}0.125 (-0.4)^2 \\
        &= 4.71
\end{aligned}$$